In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
import math

seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID, sample in the middle, static vacuum @ RT.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)


       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    9.270000e+02    9.270000e+02     9.270000e+02     9.270000e+02   
mean     1.813686e-11    5.096757e-13     3.095347e-13     6.435764e-13   
std      9.488845e-13    2.286680e-14     3.853445e-14     1.084669e-13   
min      1.752650e-11    4.320000e-13     2.542000e-13     5.521000e-13   
25%      1.787940e-11    5.017000e-13     2.933000e-13     6.150000e-13   
50%      1.806350e-11    5.082000e-13     3.056000e-13     6.337000e-13   
75%      1.826455e-11    5.150500e-13     3.210000e-13     6.556000e-13   
max      4.446610e-11    8.895000e-13     1.129200e-12     3.707700e-12   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     9.270000e+02     9.270000e+02     9.270000e+02     9.270000e+02   
mean      1.040185e-12     2.280028e-12     3.549331e-14     2.237312e-13   
std       9.860014e-14     2.641369e-13     1.930125e-14     3.384464e-13   
min       9.2800

,mass 2.0_time,mass 4.0_time,mass 12.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 4","Elapsed time, mass 12","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2022/11/17 16:09:52.405,2022/11/17 16:09:52.655,2022/11/17 16:09:52.905,2022/11/17 16:09:53.155,2022/11/17 16:09:53.405,2022/11/17 16:09:53.655,2022/11/17 16:09:53.906,2022/11/17 16:09:54.156,2022/11/17 16:09:54.406,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2022/11/17 16:09:54.728,2022/11/17 16:09:54.978,2022/11/17 16:09:55.228,2022/11/17 16:09:55.479,2022/11/17 16:09:55.729,2022/11/17 16:09:55.979,2022/11/17 16:09:56.229,2022/11/17 16:09:56.479,2022/11/17 16:09:56.730,2.323,2.323,2.323,2.324,2.324,2.324,2.323,2.323,2.324
4,2022/11/17 16:09:57.051,2022/11/17 16:09:57.301,2022/11/17 16:09:57.552,2022/11/17 16:09:57.802,2022/11/17 16:09:58.052,2022/11/17 16:09:58.302,2022/11/17 16:09:58.552,2022/11/17 16:09:59.053,NaN,4.646,4.646,4.647,4.647,4.647,4.647,4.646,4.897,NaN
5,2022/11/17 16:09:59.374,2022/11/17 16:09:59.624,2022/11/17 16:09:59.874,2022/11/17 16:10:00.124,2022/11/17 16:10:00.374,2022/11/17 16:10:00.625,2022/11/17 16:10:00.875,2022/11/17 16:10:01.375,NaN,6.969,6.969,6.969,6.969,6.969,6.970,6.969,7.219,NaN
6,2022/11/17 16:10:01.696,2022/11/17 16:10:01.946,2022/11/17 16:10:02.197,2022/11/17 16:10:02.447,2022/11/17 16:10:02.697,2022/11/17 16:10:02.947,2022/11/17 16:10:03.197,2022/11/17 16:10:03.448,2022/11/17 16:10:03.698,9.291,9.291,9.292,9.292,9.292,9.292,9.291,9.292,9.292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,2022/11/17 16:45:33.751,2022/11/17 16:45:34.001,2022/11/17 16:45:34.252,2022/11/17 16:45:34.502,2022/11/17 16:45:34.752,2022/11/17 16:45:35.002,2022/11/17 16:45:35.252,2022/11/17 16:45:35.503,2022/11/17 16:45:35.753,2141.346,2141.346,2141.347,2141.347,2141.347,2141.347,2141.346,2141.347,2141.347
925,2022/11/17 16:45:36.074,2022/11/17 16:45:36.324,2022/11/17 16:45:36.574,2022/11/17 16:45:36.824,2022/11/17 16:45:37.074,2022/11/17 16:45:37.325,2022/11/17 16:45:37.575,2022/11/17 16:45:37.825,2022/11/17 16:45:38.075,2143.669,2143.669,2143.669,2143.669,2143.669,2143.670,2143.669,2143.669,2143.669
926,2022/11/17 16:45:38.396,2022/11/17 16:45:38.646,2022/11/17 16:45:38.897,2022/11/17 16:45:39.147,2022/11/17 16:45:39.397,2022/11/17 16:45:39.647,2022/11/17 16:45:39.897,2022/11/17 16:45:40.147,2022/11/17 16:45:40.398,2145.991,2145.991,2145.992,2145.992,2145.992,2145.992,2145.991,2145.991,2145.992
927,2022/11/17 16:45:40.719,2022/11/17 16:45:40.969,2022/11/17 16:45:41.219,2022/11/17 16:45:41.469,2022/11/17 16:45:41.719,2022/11/17 16:45:41.970,2022/11/17 16:45:42.220,2022/11/17 16:45:42.470,2022/11/17 16:45:42.720,2148.314,2148.314,2148.314,2148.314,2148.314,2148.315,2148.314,2148.314,2148.314


### Position of the EGA

 

<div>
<img src="20221114_144204.jpg" width="1000"/>
</div>

In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,17/11/2022 16:09:45,7.0,-0.062096,2.190000e-09,2.510000e-09,4.996000e-09,5.749000e-07,0.000484,249.0,-5.050,4326.919,NaN,NaN
2,NaN,17/11/2022 16:09:52,14.0,-0.062764,2.180000e-09,2.500000e-09,4.996000e-09,5.749000e-07,0.000484,249.0,-5.050,4327.031,NaN,NaN
3,NaN,17/11/2022 16:10:00,22.0,-0.062492,2.180000e-09,2.480000e-09,4.996000e-09,5.756000e-07,0.000482,249.0,-5.275,4327.592,NaN,NaN
4,NaN,17/11/2022 16:10:07,29.0,-0.062147,2.170000e-09,2.480000e-09,4.996000e-09,5.756000e-07,0.000484,249.0,-5.275,4327.143,NaN,NaN
5,NaN,17/11/2022 16:10:14,36.0,-0.062343,2.160000e-09,2.470000e-09,4.996000e-09,5.791000e-07,0.000452,249.0,-5.163,4326.919,NaN,NaN


In [4]:
#Reading in the HiVolta measurement file to get the grid current
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.0,1670.000000,1.670000e+03,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1670.000000,1.670000e+03,1.670000e+03
mean,1003.899420,124.880635,93.616036,932.446922,931.823892,931.151641,0.0,0.014455,2.000000e-02,-0.058632,0.450570,29.555424,29.584670,29.562427,-0.005367,-0.016313,-0.007140,7.115461e-08,4.505701e-07
std,580.320705,215.443949,24.510796,249.446127,250.655764,251.840868,0.0,0.008955,2.255816e-16,0.071030,0.343508,7.935865,7.952343,7.958544,0.000522,0.001010,0.000998,5.847315e-08,3.435077e-07
min,1.188000,0.020000,0.060000,0.000000,0.000000,0.000000,0.0,0.000000,2.000000e-02,-0.748100,0.006100,-0.045900,-0.048500,-0.047400,-0.007300,-0.017600,-0.009600,3.000000e-10,6.100000e-09
25%,501.784000,0.080000,100.040000,1000.000000,1000.100000,1000.120000,0.0,0.000000,2.000000e-02,-0.094500,0.019600,31.531325,31.495600,31.543300,-0.005900,-0.017400,-0.007700,1.630000e-08,1.960000e-08
50%,1002.609500,0.100000,100.060000,1000.010000,1000.120000,1000.160000,0.0,0.020000,2.000000e-02,-0.080200,0.549950,31.870200,31.952650,31.923400,-0.005300,-0.016400,-0.007300,8.050000e-08,5.499500e-07
75%,1505.909500,198.970000,100.060000,1000.020000,1000.140000,1000.160000,0.0,0.020000,2.000000e-02,0.003000,0.640525,31.990175,32.036050,32.015075,-0.004800,-0.015600,-0.006500,9.520000e-08,6.405250e-07
max,2008.877000,499.960000,100.140000,1000.040000,1000.160000,1000.160000,0.0,0.020000,2.000000e-02,0.511400,3.242600,32.268700,32.246700,32.236200,-0.004600,-0.014500,-0.005700,7.481000e-07,3.242600e-06


### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="emission on"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 11, 17, 16, 12, 48)
y=pd.Timestamp(2022, 11, 17, 16, 28, 1)

#then find the time value closest to "emission on" in the MID scan file 
print("\n","Result 3:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])

#then find the time value closest to "grid repelling" in the MID scan file 
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 5:",bck)

#remove values under 10^-14 from mid
mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck
print("Res 6:",mid["mass 2, corr"].describe())

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][78:471]/h2_sens
print("Result 7:",p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:78]/h2_sens
print("Result 8:",base_h2.describe())

Result 1:            Date          Time  Elapsed time  VMon1   VMon2   VMon3   VMon4  \
121  17/11/2022  16:12:48.771       145.234   0.06  100.10  999.72  999.74   
122  17/11/2022  16:12:49.990       146.453   0.08  100.10  999.72  999.74   
123  17/11/2022  16:12:51.180       147.643   0.08  100.10  999.72  999.74   
124  17/11/2022  16:12:52.373       148.836   0.08  100.08  999.74  999.74   

      VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
121  999.76    0.0   0.02  ...  0.7380  29.7435  29.7656  29.7564 -0.0059   
122  999.76    0.0   0.02  ...  0.7304  29.7531  29.7573  29.7517 -0.0059   
123  999.76    0.0   0.02  ...  0.7307  29.7616  29.7588  29.7593 -0.0059   
124  999.76    0.0   0.02  ...  0.7353  29.7710  29.7678  29.7684 -0.0059   

      IMon7  IMon8      Comment       hv_grid            em  
121 -0.0157 -0.008  emission on  9.300000e-08  7.380000e-07  
122 -0.0157 -0.008  emission on  9.520000e-08  7.304000e-07  
123 -0.0157 -0.008  emissio

Res 6: count    9.270000e+02
mean     1.791313e-11
std      9.488845e-13
min      1.730277e-11
25%      1.765567e-11
50%      1.783977e-11
75%      1.804082e-11
max      4.424237e-11
Name: mass 2, corr, dtype: float64
Result 7: count    3.930000e+02
mean     8.402740e-10
std      9.219205e-12
min      8.215760e-10
25%      8.340173e-10
50%      8.387591e-10
75%      8.453694e-10
max      8.688624e-10
Name: mass 2, corr, dtype: float64
Result 8: count    7.800000e+01
mean     8.629585e-10
std      9.421791e-12
min      8.483506e-10
25%      8.546593e-10
50%      8.619234e-10
75%      8.698612e-10
max      8.874633e-10
Name: mass 2, corr, dtype: float64


C:\Users\etiirine\AppData\Local\Temp\ipykernel_16628\2655486023.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 40.0_value"][mid["mass 40.0_value"] < 1e-14] = np.nan


### Plotting

In [16]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan, EGA in the middle on a-C @ 4K, static vacuum")
plt.yscale('log')

In [15]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="best")
plt.title("HiVolta current recording, EGA in the middle on a-C @ 4K, static vacuum")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 14,
        }
plt.text(300, 1e-6, 'Emission on', fontdict = font)
plt.text(1060, 7e-9, 'Grid repelling', fontdict = font)
plt.text(1300, 1e-6, 'Grid transparent', fontdict = font)
plt.text(1490, 7e-9, 'Grid repelling, EGA turned', fontdict = font)
plt.text(1780, 1e-6, 'Grid transparent', fontdict = font)

plt.yscale('log')


In [9]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution, EGA in the middle on a-C @ 4K, static vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
ax.yaxis.set_minor_formatter(LogFormatter(minor_thresholds=(2,0.5)))
plt.show()

In [10]:
#Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution, EGA in the middle on a-C @ 4K, static vacuum")
plt.yscale('linear')